In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy import signal
import mat73

In [2]:
import h5py

filepath = './data/sub07.mat'
arrays = {}
f = h5py.File(filepath)
for k, v in f.items():
    arrays[k] = np.array(v)

In [ ]:
data_mat = mat73.loadmat('./data/sub07.mat')

In [ ]:
data_mat

In [ ]:
print(type(data_mat))
print(data_mat.keys())
print(data_mat['data'].keys())

In [ ]:
data_mat['data']['file2'].shape
data_mat['data']['file2']

In [ ]:
df = pd.DataFrame(data_mat['data']['file2'], columns=['x_coordinate', 'y_coordinate', 'Picture Number', 'Reaction Time (RT)'])
df['Mark for Picture Shown'] = data_mat['data']['mark2'][0::2]
df['Mark for Picture Placed'] = data_mat['data']['mark2'][1::2]
df.head()

In [ ]:
df['Timestamp (s) for Picture Shown'] = df['Mark for Picture Shown'].apply(lambda x: x/fs)
df['Timestamp (s) for Picture Placed'] = df['Mark for Picture Placed'].apply(lambda x: x/fs)
df.head()

In [ ]:
idx = df['Picture Number'][df['Picture Number'] == 43].index.to_list()
df.iloc[idx]

In [ ]:
data_mat['data']['ieeg_data2'].shape

In [ ]:
#******** Variable definitions ********
ieeg_contact = 0                                    # iEEG contact
roi = [9, 13]                                       # ROI in seconds

#******** Static definitions ********
fs = 512                                            # sample frequency
TT = data_mat['data']['ieeg_data2'].shape[1] / fs   # total time in seconds
t = np.arange(0, TT, 1/fs)                          # full time vector
t_roi = np.arange(roi[0], roi[1], 1/fs)             # ROI time vector

#******** Plotting ********
fig, ax = plt.subplots(2, 1, figsize=(20, 15))
ax[0].plot(t, data_mat['data']['ieeg_data2'][ieeg_contact, :])
ax[0].set_title('Subject 07 - ieeg_data2', fontsize=20)
ax[0].set_xlabel('Time (s)', fontsize=12)
ax[0].set_ylabel('Amplitude (uV)', fontsize=12)
ax[0].set_xlim([t[0], t[-1]])

ax[1].plot(t_roi, data_mat['data']['ieeg_data2'][ieeg_contact, roi[0]*fs:roi[1]*fs])
ax[1].set_title('Subject 07 - ieeg_data2', fontsize=20)
ax[1].set_xlabel('Time (s)', fontsize=12)
ax[1].set_ylabel('Amplitude (uV)', fontsize=12)
ax[1].set_xlim([t_roi[0], t_roi[-1]])
plt.tight_layout()

## Filter Signal

In [ ]:
fs = 512 # sample frequency
t = np.linspace(0, 1, sr, False)  # 1 second
sig = np.sin(2*np.pi*2*t) + np.sin(2*np.pi*10*t) + np.sin(2*np.pi*50*t)
order = 4
cutoff = [5, 15]
filter_type = 'bandpass'
show_impulse_response = True
len_impulse = 1000

impulse = signal.unit_impulse(len_impulse)
t_impulse = np.arange(-10, len_impulse-10)

# Create digital filter
b, a = signal.butter(N=order, Wn=cutoff, btype=filter_type, fs=fs, analog=False)
# Compute digital filter frequency response
w, h = signal.freqz(b, a, fs=fs)
out_signal = signal.lfilter(b, a, sig)

plt.figure(figsize=(12,7))
plt.semilogx(w, 20*np.log10(abs(h)), linewidth=3)
plt.title(f"Digital Filter Frequency Response - {filter_type.upper()}", fontsize=22)
plt.xlabel("Frequency [Hz]", fontsize=16)
plt.ylabel("Amplitude [dB]", fontsize=16)
plt.grid(which='both', axis='both')
if filter_type in ('low', 'lowpass'): plt.margins(0, 0.15)
if filter_type in ('high', 'highpass'): plt.margins(0.15, 0.1)
if filter_type in ('bs', 'bandstop', 'bp', 'bandpass'): 
    plt.axvline(cutoff[0], color='orange', label=f"Cutoff frequency start ({cutoff[0]} Hz)", linewidth=2, linestyle='--')
    plt.axvline(cutoff[1], color='orange', label=f"Cutoff frequency stop ({cutoff[1]} Hz)", linewidth=2, linestyle='--')
if filter_type not in ('bs', 'bandstop', 'bp', 'bandpass'): 
    plt.semilogx(cutoff, 20*np.log10(0.5*np.sqrt(2)), '*') # plot a star symbol at -3 dB attenuation
    plt.axvline(cutoff, color='orange', label=f"Cutoff frequency ({cutoff} Hz)", linewidth=2, linestyle='--')
plt.legend(loc='best', fontsize=12, shadow=True)

if show_impulse_response:
    response = signal.lfilter(b, a, impulse)
    
    plt.figure(figsize=(12,7))
    plt.plot(t_impulse, impulse, 'b-', label='Impulse')
    plt.plot(t_impulse, response, 'g-', linewidth=2, label='Response')
    plt.xlabel('Time [sec]', fontsize=16)
    plt.ylabel('Amplitude', fontsize=16)
    plt.grid(which='both', axis='both')
    plt.legend(loc='best', fontsize=12, shadow=True)

plt.show()

# plot
fig, ax = plt.subplots(2, 1, figsize=(20, 10), sharex=True)
ax[0].plot(t, sig)
ax[0].set_title('Original Signal', fontsize=20)
ax[0].set_xlabel('Time (s)', fontsize=12)
ax[0].set_ylabel('Amplitude (V)', fontsize=12)

ax[1].plot(t, out_signal)
ax[1].set_title('Detrended Signal', fontsize=20)
ax[1].set_xlabel('Time (s)', fontsize=12)
ax[1].set_ylabel('Amplitude (V)', fontsize=12)
plt.tight_layout()

In [ ]:
#******** Variable definitions ********
ieeg_contact = 0                                            # single iEEG contact to inspect
roi = [9, 13]                                               # ROI in seconds

#******** Static definitions ********
fs = 512                                                    # sample frequency
TT = Data.dataone.shape[1] / fs  # total time in seconds
t = np.arange(0, TT, 1/fs)                                  # full time vector
t_roi = np.arange(roi[0], roi[1], 1/fs)                     # ROI time vector

#******** Fourier transform ********
y = fft(Data.dataone[ieeg_contact, :])                      # FFT
xf = fftfreq(Data.dataone[ieeg_contact, :].size, 1/fs)      # Frequency vector
xf = xf[xf>=0]                                              # Only positive frequencies
y = np.abs(y)[:len(xf)]                                     # Magnitude of FFT (one sided)


#******** Plotting ********
fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(20, 20))

# Entire signal
ax[0].plot(t, Data.dataone[ieeg_contact, :])
ax[0].set_title('Raw iEEG', fontsize=20)
ax[0].set_xlabel('Time (s)', fontsize=12)
ax[0].set_ylabel('Amplitude (uV)', fontsize=12)
ax[0].set_xlim([t[0], t[-1]])

# ROI
ax[1].plot(t_roi, Data.dataone[ieeg_contact, roi[0]*fs:roi[1]*fs])
ax[1].set_title('iEEG ROI', fontsize=20)
ax[1].set_xlabel('Time (s)', fontsize=12)
ax[1].set_ylabel('Amplitude (uV)', fontsize=12)
ax[1].set_xlim([t_roi[0], t_roi[-1]])

# Magnitude spectrum - Entire signal
ax[2].plot(xf, y, color='orange')
ax[2].set_title('Magnitude Spectrum', fontsize=20)
ax[2].set_xlabel('Frequency (Hz)', fontsize=12)
ax[2].set_ylabel('Magnitude', fontsize=12)
ax[2].set_xlim([xf[0], xf[-1]])

# Spectrogram
pxx, freqs, _t, img = ax[3].specgram(Data.dataone[ieeg_contact, :], Fs=fs, NFFT=512, noverlap=64, cmap='rainbow')
ax[3].set_title('Spectrogram', fontsize=20)
ax[3].set_xlabel('Time (s)', fontsize=12)
ax[3].set_ylabel('Frequency (Hz)', fontsize=12)

fig.suptitle(f"Experiment Phase {experiment_phase_of_interest} - Subject {Data.subject_id}", fontsize=24, fontweight='bold')
plt.tight_layout()

In [ ]:
def digital_filter(order=4, cutoff=500, fs=1000, filter_type='high', apply_filter=False, in_signal=None):
    """digital_filter Create a digital filter using scipy's butter function.

    Create a digital filter using scipy's butter function and compute the
    filter frequency response. The filter frequency response is always plotted
    as part in ensuring the desired filter design.

    Args:
        order (int, optional): The order of the filter. Defaults to 4.
        cutoff (int, optional): The cutoff frequency of the filter (in Hz). Defaults to 500.
        fs (int, optional): The sampling frequency used to sample the signal. Defaults to 1000.
        filter_type (str, optional): The type of filter ('low', 'lowpass', 'high', 'highpass'). Defaults to 'high'.
        apply_filter (bool, optional): If True then the filter is applied to the signal in_signal. Defaults to False.
        in_signal (_type_, optional): If specified and apply_filter is set to True, the filter is applied to the signal 
                                      and the filtered signal is returned. Defaults to None.

    Returns:
        b, a (ndarray): Default setting for function is to return the 
                        filter coefficients.
        out_signal (ndarray): If apply_filter is set to True and in_signal 
                              is given, the function will apply the digital 
                              filter and return the filtered signal.
    """
    # Create digital filter
    b, a = signal.butter(N=order, Wn=cutoff, btype=filter_type, fs=fs, analog=False)
    
    # Compute digital filter frequency response
    w, h = signal.freqz(b, a, fs=fs)
    
    # Plot a bode plot of filter frequency response
    plt.figure(figsize=(12,7))
    plt.semilogx(w, 20*np.log10(abs(h)), linewidth=3)
    plt.semilogx(cutoff, 20*np.log10(0.5*np.sqrt(2)), '*') # plot a star symbol at -3 dB attenuation
    plt.title("Digital Filter Frequency Response", fontsize=22)
    plt.xlabel("Frequency [Hz]", fontsize=16)
    plt.ylabel("Amplitude [dB]", fontsize=16)
    plt.grid(which='both', axis='both')
    if filter_type in ('low', 'lowpass'): plt.margins(0, 0.15)
    if filter_type in ('high', 'highpass'): plt.margins(0.15, 0.1)
    plt.axvline(cutoff, color='orange', label=f"Cutoff Frequency ({cutoff} Hz)", linewidth=2, linestyle='--')
    plt.legend(loc='best', fontsize=12, shadow=True)
    plt.show()
    
    if apply_filter == True and in_signal is not None:
        # Apply filter and return filtered signal
        out_signal = signal.lfilter(b, a, in_signal)
        return out_signal
    else:
        # (Defualt) Return the filter coefficients
        return b, a

In [ ]:
data_filtered = digital_filter(in_signal=data_mat['data']['ieeg_data2'][0, :],
                                order=6,
                                cutoff=1,
                                filter_type='highpass',
                                fs=fs,
                                apply_filter=True)

In [ ]:
#******** Plotting ********
fig, ax = plt.subplots(2, 1, figsize=(20, 15))
ax[0].plot(t_roi, data_mat['data']['ieeg_data2'][ieeg_contact, roi[0]*fs:roi[1]*fs])
ax[0].set_title('Subject 07 - Unfiltered', fontsize=20)
ax[0].set_xlabel('Time (s)', fontsize=12)
ax[0].set_ylabel('Amplitude (uV)', fontsize=12)
ax[0].set_xlim([t_roi[0], t_roi[-1]])

ax[1].plot(t_roi, data_filtered[roi[0]*fs:roi[1]*fs])
ax[1].set_title('Subject 07 - Filtered', fontsize=20)
ax[1].set_xlabel('Time (s)', fontsize=12)
ax[1].set_ylabel('Amplitude (uV)', fontsize=12)
ax[1].set_xlim([t_roi[0], t_roi[-1]])
plt.tight_layout()

## Power Spectral Analyis

In [ ]:
f, t, Sxx = signal.spectrogram(data_mat['data']['ieeg_data2'][0, :], fs)
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
f, Pxx_den = signal.periodogram(data_filtered, fs)
plt.semilogy(f, Pxx_den)